In [25]:
import os
import psycopg
import pandas as pd

In [26]:
postgres_credentials = {'dbname': os.getenv("DB_DESTINATION_NAME"),
                            'host': os.getenv("DB_DESTINATION_HOST"),
                           'password': os.getenv("DB_DESTINATION_PASSWORD"),
                           'port': os.getenv("DB_DESTINATION_PORT"),
                           'user': os.getenv("DB_DESTINATION_USER")}

In [27]:
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])
connection.update(postgres_credentials)

    # определим название таблицы, в которой хранятся наши данные.
TABLE_NAME = "flat_dataset"

    # эта конструкция создаёт контекстное управление для соединения с базой данных 
    # оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций 
    # закрыто оно будет даже в случае ошибки, чтобы не допустить "утечку памяти"
with psycopg.connect(**connection) as conn:

        # создаёт объект курсора для выполнения запросов к базе данных
        # с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT *  FROM {TABLE_NAME}")     
        # извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()
        # получает список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]

    # создаёт объект DataFrame из полученных данных и имён столбцов. 
    # это позволяет удобно работать с данными в Python, используя библиотеку Pandas.
df = pd.DataFrame(data, columns=columns) 

In [28]:
df

,id,flat_id,floor,is_apartment,kitchen_area,living_area,rooms,studio,total_area,price,build_year,building_type_int,ceiling_height,flats_count,floors_total,has_elevator,nearest_metro,distance_to_metro
0,21820,23804,1,false,6.0,27.000000,2,false,44.400002,12500000,1961,4,2.64,80,5,false,Ломоносовский проспект,5.684752
1,21821,23805,3,false,6.5,21.600000,1,false,35.500000,8000000,1974,4,2.64,404,9,true,Новогиреево,9.158784
2,21822,23806,24,false,9.7,18.000000,1,false,39.400002,8800000,2002,2,2.78,143,24,true,Шоссе Энтузиастов,10.352290
3,21823,23807,1,false,8.3,41.400002,3,false,71.000000,12400000,2012,4,2.70,487,19,true,ЦСКА,29.702125
4,21824,23808,4,false,8.0,17.000000,1,false,34.099998,8500000,2018,2,2.80,399,9,true,Мичуринский проспект,16.163949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127039,77239,85426,12,false,9.9,19.900000,1,false,35.099998,10150000,1965,6,2.64,96,12,true,Шоссе Энтузиастов,5.529926
127040,77240,85427,14,false,8.5,19.000000,1,false,38.000000,7100000,1995,4,2.64,133,17,true,Марксистская,13.084415
127041,77241,85428,4,false,14.1,59.299999,3,false,93.199997,18900000,2017,2,3.00,557,9,true,Рассказовка,0.951052
127042,77242,85429,11,false,10.0,18.000000,1,false,35.799999,8200000,2009,4,2.64,128,17,true,Петровский парк,5.793875


In [32]:
import joblib

# Загружаем модель из DVC
pipeline = joblib.load('../models/fitted_model.pkl')

print(pipeline[1])

In [33]:
import json
# Открываем файл json
with open('../cv_results/cv_res.json', 'r') as file:
    metrix = json.load(file)

print(metrix)

{'fit_time': 33.567, 'score_time': 0.097, 'test_neg_mean_absolute_error': -2046508.741, 'test_neg_root_mean_squared_error': -2665176.755, 'test_r2': 0.716, 'test_neg_mean_absolute_percentage_error': -0.179}


In [34]:
params = pipeline[1].get_params(deep=False)

In [35]:
import mlflow
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = 'baseline'
RUN_NAME = "baseline"
REGISTRY_MODEL_NAME = 'baseline'
FS_ASSETS = "baseline"  

In [36]:
# устанавливаем host, который будет отслеживать наши эксперименты
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}") 

In [39]:
mlflow.set_experiment(EXPERIMENT_NAME)
experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
with mlflow.start_run(run_name=f"{RUN_NAME}_intersection_and_union", experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    mlflow.log_params(params)
    mlflow.log_metrics(metrix)
    metadata = {"model_type": "baseline", 'structure': 'Catboost', 'train_type': 'pipeline'}
    train_data = df.drop(['price','flat_id','id'], axis=1)
    train_data =train_data.head(5)
    pred = pipeline.predict(train_data)
    signature = mlflow.models.infer_signature(train_data, pred)
    input_example = df.iloc[:3]
    pip_requirements = 'yn/requirements.txt'
    model_info = mlflow.pyfunc.log_model( 
		    python_model=pipeline,
            artifact_path="models",
            registered_model_name=REGISTRY_MODEL_NAME,
            signature=signature,
            input_example=input_example,
            pip_requirements=pip_requirements,
            metadata = metadata,
            await_registration_for=60
		    )
    #mlflow.pyfunc.log_model(artifact_path="model",python_model=ETS_Exogen, conda_env=conda_env)
    mlflow.end_run()
    

/home/mle-user/mle_projects/.venv/lib/python3.10/site-packages/mlflow/models/signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None


MlflowException: `python_model` must be a PythonModel instance or a callable object